In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np

import random
import h5py
from keras.datasets import cifar10
from keras.models import *
from keras.layers import *
from keras.layers.core import *
from keras.layers.normalization import *
from keras.optimizers import *
from keras.callbacks import *
from keras import backend as K
from keras.regularizers import *
import theano.tensor as T
import theano
from theano.tensor.shared_randomstreams import RandomStreams
from sklearn import metrics
from skimage.measure import compare_ssim
from scipy.misc import toimage
from sklearn.preprocessing import *

import os
import random
import time
from skimage import io, exposure, feature, color, transform
import matplotlib
import matplotlib.pyplot as plt
import glob

import scipy.signal as sig
import operator
import math
import re

# for reproducibility
np.random.seed(1337) 
random.seed(1337)

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 690 (CNMeM is disabled, cuDNN 5105)


In [2]:
# parameters for sliding window, and window function (Hann)
STEP_SIZE = 480
OVERLAP_SIZE = 32
WINDOW_SIZE = STEP_SIZE + OVERLAP_SIZE
OVERLAP_FUNC = sig.hann(OVERLAP_SIZE * 2)

# directory that contains TIMIT files
TIMIT_DIR = "/home/sri/Desktop/timit"

# randomly shuffle data before partitioning into training/validation?
RANDOM_SHUFFLE = True

# sample rate of input file (used in MFCC calculation)
SAMPLE_RATE = 16000

In [3]:
from load_TIMIT import *
from windowingFunctions import *
from utility import *

In [4]:
# read in 100 WAVs from TIMIT training set
rawWaveforms = load_TIMIT_train(TIMIT_DIR, 1000)

Reading in .wav files...


In [5]:
# waveform preprocessing
def preprocessWaveform(waveform):
    # scale window between -1 and 1
    mn = np.min(waveform)
    mx = np.max(waveform)
    maxabs = np.maximum(np.abs(mn), np.abs(mx))
        
    return np.copy(waveform) / maxabs, (maxabs,)
   
def unpreprocessWaveform(waveform, params):
    return np.copy(waveform) * params[0]



# window preprocessing
def preprocessWindows(windows):
    return windows, ()

def unpreprocessWindows(windows, params):
    return windows

In [6]:
# waveform preprocessing
processedWaveforms = np.copy(rawWaveforms)

# we maximize the volume of every waveform
for i in xrange(0, len(processedWaveforms)):
    processedWaveforms[i], _ = preprocessWaveform(rawWaveforms[i])


In [7]:
# extract windows
rawWindows = extractWindowsMultiple(processedWaveforms, STEP_SIZE, OVERLAP_SIZE,
                                    collapse = True)

# randomly shuffle data
if (RANDOM_SHUFFLE):
    rawWindows = np.random.permutation(rawWindows)

print "Raw windows shape: ", rawWindows.shape
print "Max: ", np.amax(rawWindows)
print "Min: ", np.amin(rawWindows)

Raw windows shape:  (101135, 512)
Max:  1.0
Min:  -1.0


In [8]:
# data augmentation goes here, at some point
augWindows = np.copy(rawWindows)

print "Aug windows shape: ", augWindows.shape

Aug windows shape:  (101135, 512)


In [9]:
processedWindows, pwParams = preprocessWindows(augWindows)

In [10]:
# reshape into vector form
processedWindows = np.reshape(processedWindows, (processedWindows.shape[0], WINDOW_SIZE, 1))

In [11]:
print processedWindows.shape

print np.mean(processedWindows, axis=None)
print np.std(processedWindows, axis=None)
print np.min(processedWindows, axis = None)
print np.max(processedWindows, axis = None)

(101135, 512, 1)
9.77603e-07
0.099766
-1.0
1.0


In [12]:
class CodeRound(T.Op):
    # properties attribute
    __props__ = ()
    
    def __init__(self, nbins):
        self.nbins = nbins
        super(CodeRound, self).__init__()
        
    def make_node(self, x):
        assert hasattr(self, '_props'), "Your version of theano is too old to support __props__."
        x = T.as_tensor_variable(x)
        return theano.Apply(self, [x], [x.type()])
    
    def perform(self, node, inputs, output_storage):
        x, = inputs
        z, = output_storage
        
        s = (x + 1.0) / 2.0
        s = np.round(s * float(self.nbins - 1)) / float(self.nbins - 1)
        s = (s * 2.0) - 1.0
        
        z[0] = s
    
    def grad(self, input, output_gradients):
        # pass through gradients unchanged
        x, = input
        g, = output_gradients
        return [g]
        
    def infer_shape(self, node, i0_shapes):
        # output shape is same as input shape
        return i0_shapes

In [13]:
class PhaseShiftUp1D(Layer):
    """ PhaseShiftUp1D
    Takes vector of size: B x S x nF
    And returns vector: B x nS x F
    """
    def __init__(self, n, **kwargs):
        super(PhaseShiftUp1D, self).__init__(**kwargs)
        self.n = n
    
    def build(self, input_shape):
        # no trainable parameters
        self.trainable_weights = []
        super(PhaseShiftUp1D, self).build(input_shape)
        
    def call(self, x, mask=None):
        r = T.reshape(x, (x.shape[0], x.shape[1], x.shape[2] / self.n, self.n))
        r = T.transpose(r, (0, 1, 3, 2))
        r = T.reshape(r, (x.shape[0], x.shape[1] * self.n, x.shape[2] / self.n))
        return r

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1] * self.n, input_shape[2] / self.n)
    
    def get_config(self):
        config = {'n' : self.n}
        base_config = super(PhaseShiftUp1D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [14]:
def linear_upsample_1d(x):
    import theano.tensor as T
    
    r = T.repeat(x, 2, axis = 1)
    s = T.roll(r, -1, axis = 1)
    u = ((r[:, :-1] + s[:, :-1]) / 2.0)
    u = T.concatenate((u, r[:, -1:]), axis = 1)
    return u

def linear_upsample_shape(shape):
    return (shape[0], shape[1] * 2, shape[2])

# linear upsampling "layer"
def LinearUpSampling1D():
    return Lambda(linear_upsample_1d, output_shape = linear_upsample_shape)

In [15]:
# ------------------------------------------------------------------
# RMSE LOSS FUNCTION (NaN-safe)
# ------------------------------------------------------------------
def rmse(y_true, y_pred):
    mse = K.mean(K.square(y_pred - y_true), axis=-1)
    return K.sqrt(mse + K.epsilon())

In [16]:
# ------------------------------------------------------------------
# THEANO DFT FUNCTIONS / FREQUENCY LOSS
# ------------------------------------------------------------------

# generate square dft matrix (similar to how we generate the DFT one)
#     note that this matrix will have real and imaginary components
def generate_dft_mat(n):
    return (np.fft.fft(np.eye(n)))

# we compute both the real and imaginary part of the FFT separately, at program start
dftMat = generate_dft_mat(WINDOW_SIZE)
th_dftMat_imag = K.variable(np.imag(dftMat))
th_dftMat_real = K.variable(np.real(dftMat))

# given a (symbolic Theano) array of size M x WINDOW_SIZE (x 1)
#     this returns an array M x WINDOW_SIZE x 2 where every one of the M samples has been
#     its DFT magnitude
def theano_dft(x):
    global th_dftMat_imag
    global th_dftMat_real

    reshaped_x = x.reshape((1, x.shape[0], x.shape[1]))

    imag = T.tensordot(th_dftMat_imag, reshaped_x, [[0], [2]])
    imag = imag.reshape((1, imag.shape[0], imag.shape[2])).T

    real = T.tensordot(th_dftMat_real, reshaped_x, [[0], [2]])
    real = real.reshape((1, real.shape[0], real.shape[2])).T
    
    dft = K.concatenate([real, imag], axis = 2)
    return dft

# given a (symbolic Theano) array of size M x WINDOW_SIZE (x 1)
#     this returns an array M x WINDOW_SIZE where every one of the M samples has been replaced by
#     its DFT magnitude
def theano_dft_mag(x):
    dft = theano_dft(x)
    
    real = dft[:, :, 0]
    imag = dft[:, :, 1]
    
    result = K.sqrt(K.square(real) + K.square(imag) + K.epsilon())
    return result

# return loss over upper frequencies of DFT (2000Hz+)
def upper_freq_loss(y_true, y_pred):
    dft_true = theano_dft(y_true)
    dft_pred = theano_dft(y_pred)
    
    start = WINDOW_SIZE * (2000.0 / 16000.0)
    start = int(start)
    
    upper_true = dft_true[:, start:]
    upper_pred = dft_pred[:, start:]
    
    return K.mean(K.abs(upper_true - upper_pred), axis = -1)

In [17]:
# freeze weights for stacked training
def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [18]:
# ====================================================
# PARAMETERS FOR AUTOENCODER STRUCTURE
# ====================================================
NBINS = 201
TIMES_DOWNSAMPLE = 2

NCHAN = 32
FILT_SIZE = 7
FILT_MID = FILT_SIZE / 2 + 1

NUM_RES_BLOCKS = 4

input_dim = (WINDOW_SIZE, 1)
input_size = np.prod(input_dim)
bottleneck_size = WINDOW_SIZE / int(2 ** TIMES_DOWNSAMPLE)

res_init = 'glorot_normal'

def activation():
    return LeakyReLU(0.3)
    #return ELU()
    #return PReLU()
    #return Activation('relu')

In [19]:

# ----------------------------------------------------
# weight initializers
# ----------------------------------------------------

# convolutional layers have weight matrices of shape:
#     (filter_length, input_dim, nb_filter)
# and biases of size (nb_filter,)

# weights for an "replication convolution" (which just
# replicates the input across all channels)
#     filter of length FILT_SIZE, going from 1 channel to NCHAN
def replicate_conv():
    weights = np.zeros((FILT_SIZE, 1, NCHAN))
    weights[FILT_MID] = np.ones((1, NCHAN))
    #weights += np.random.uniform(-0.005, 0.005, weights.shape)
    biases = np.zeros((NCHAN,))
    return [weights, biases]

# weights for an "average convolution" (which just
# replicates the input across all channels)
#     filter of length 1, going from NCHAN channels to 1
def average_conv(sz):
    mid = sz / 2 + 1
    if (sz == 1): mid = 0

    weights = np.zeros((sz, NCHAN, 1))
    weights[mid] = np.ones((NCHAN, 1)) / float(NCHAN)
    #weights += np.random.uniform(-0.005, 0.005, weights.shape)
    biases = np.zeros((1,))
    return [weights, biases]

# weights for a "identity" convolution (by default,
# does nothing to its input)
#     filter of length FILT_SIZE, going from NCHAN to NCHAN
#     (when combined with subsampling, this is a nearest-neighbor
#      downsample)
def identity_conv():
    weights = np.zeros((FILT_SIZE, NCHAN, NCHAN))
    weights[FILT_MID] = np.eye(NCHAN)
    #weights += np.random.uniform(-0.005, 0.005, weights.shape)
    biases = np.zeros((NCHAN,))
    return [weights, biases]

# weights for a "phase shift up" convolution (by default,
# performs an upsample)
#     filter of length FILT_SIZE, going from NCHAN to NCHAN * 2
def shift_up_conv():
    weights = np.zeros((FILT_SIZE, NCHAN, NCHAN * 2))
    weights[FILT_MID] = np.repeat(np.eye(NCHAN), 2, axis = 1)
    #weights += np.random.uniform(-0.01, 0.01, weights.shape)
    biases = np.zeros((NCHAN * 2,))
    return [weights, biases]

In [20]:
# ----------------------------------------------------
# blocks of network
# ----------------------------------------------------

# residual block, going from NCHAN to NCHAN channels
def residual_block(num_chans = NCHAN):
    def f(input):
        shortcut = input
        
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear',
                     bias = True)(input)
        res = activation()(res)
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear',
                     use_bias = True)(res)
        
        m = Add()([shortcut, res])
        return m
    
    return f


# increase number of channels from 1 to NCHAN via convolution
def channel_increase_block(num_chans = NCHAN):
    def f(input):
        shortcut = Permute((2, 1))(input)
        shortcut = UpSampling1D(NCHAN)(shortcut)
        shortcut = Permute((2, 1))(shortcut)
        
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(input)
        res = activation()(res)
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(res)
        
        m = Add()([shortcut, res])
        return m
        
    return f


# downsample the signal 2x
def downsample_block(num_chans = NCHAN):
    def f(input):
        shortcut = AveragePooling1D(2)(input)
        
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear',
                     strides = 2)(input)
        res = activation()(res)
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(res)
        
        m = Add()([shortcut, res])
        return m
    
    return f


# upsample the signal 2x
def upsample_block(num_chans = NCHAN):
    def f(input):
        shortcut = UpSampling1D(2)(input)
        
        res = Conv1D(num_chans * 2, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(input)
        res = PhaseShiftUp1D(2)(res)
        res = activation()(res)
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(res)
        
        m = Add()([shortcut, res])
        return m
    
    return f


# increase number of channels from NCHAN to 1 via convolution
def channel_decrease_block(num_chans = NCHAN):
    def f(input):
        shortcut = Permute((2, 1))(input)
        shortcut = GlobalAveragePooling1D()(shortcut)
        shortcut = Reshape((-1, 1))(shortcut)
        
        res = Conv1D(num_chans, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(input)
        res = activation()(res)
        res = Conv1D(1, FILT_SIZE, padding = 'same',
                     kernel_initializer = res_init,
                     activation = 'linear')(res)

        m = Add()([shortcut, res])
        return m
        
    return f


    
# ---------------------------------------------------------------------------
# autoencoder: takes an audio window, compresses it, and tries to reconstruct it
# ---------------------------------------------------------------------------
def autoencoder_structure(dim):
    enc_input = Input(shape = dim)
    enc = Reshape(dim, input_shape = dim)(enc_input)
    
    # increase number of channels via convolution
    enc = channel_increase_block()(enc)
    
    # downsampling blocks
    for i in xrange(0, TIMES_DOWNSAMPLE): 
        enc = downsample_block()(enc)
        
    # residual blocks
    for i in xrange(0, NUM_RES_BLOCKS):
        enc = residual_block()(enc)
    
    # decrease back down to 1 channel
    enc = channel_decrease_block()(enc)
    enc = Activation('tanh')(enc)
    
    enc = Reshape((bottleneck_size,))(enc)    
    enc = Lambda(lambda x : CodeRound(NBINS)(x))(enc)
    
    enc = Model(input = enc_input, output = enc)
    
    
    
    
    
    dec_input = Input(shape = (bottleneck_size,))    
    dec = Reshape((bottleneck_size, 1))(dec_input)
    
    # increase number of channels via convolution
    dec = channel_increase_block()(dec)
    
    # residual blocks
    for i in xrange(0, NUM_RES_BLOCKS):
        dec = residual_block()(dec)
    
    # upsampling blocks
    for i in xrange(0, TIMES_DOWNSAMPLE):
        dec = upsample_block()(dec)
    
    # decrease back down to 1 channel
    dec = channel_decrease_block()(dec)
    dec = Activation('tanh')(dec)   
    
    dec = Model(input = dec_input, output = dec)
    
    return enc, dec

# ---------------------------------------------------------------------------
# discriminator: tries to differentiate between original and reconstructed samples
# ---------------------------------------------------------------------------
DSC_FILTS = 32
def discriminator_structure(dim):
    dsc_input = Input(shape = dim)
    dsc = Reshape(dim, input_shape = dim)(dsc_input)
    
    # receptive field: 139 samples
    dsc = Conv1D(DSC_FILTS, 7, strides = 2,
                 padding = 'valid',
                 kernel_initializer = res_init,
                 activation = 'linear')(dsc)
    dsc = activation()(dsc)
    
    # receptive field: 67 samples
    dsc = Conv1D(DSC_FILTS * 2, 7, strides = 2,
                 padding = 'valid',
                 kernel_initializer = res_init,
                 activation = 'linear')(dsc)
    dsc = activation()(dsc)
    
    # receptive field: 31 samples
    dsc = Conv1D(DSC_FILTS * 4, 7, strides = 2,
                 padding = 'valid',
                 kernel_initializer = res_init,
                 activation = 'linear')(dsc)
    dsc = activation()(dsc)
    
    # receptive field: 13 samples
    dsc = Conv1D(DSC_FILTS * 4, 7, strides = 1,
                 padding = 'valid',
                 kernel_initializer = res_init,
                 activation = 'linear')(dsc)
    dsc = activation()(dsc)
    
    # receptive field: 7 samples
    dsc = Conv1D(1, 7, strides = 1,
                 padding = 'valid',
                 kernel_initializer = res_init,
                 activation = 'linear')(dsc)

    # sigmoid output (probability of patch being real or fake)
    dsc = Activation('sigmoid')(dsc)
    
    # take mean over all applications of "PatchGAN"
    dsc = Flatten()(dsc)
    dsc = Lambda(lambda x : K.mean(x, axis = -1, keepdims = True),
                 lambda shape : (shape[0], 1))(dsc)

    dsc = Model(input = dsc_input, output = dsc)
    return dsc
    


# construct autoencoder to be used in adversarial training
ac_input = Input(shape = input_dim)
ac_enc, ac_dec = autoencoder_structure(input_dim)
ac_embedding = ac_enc(ac_input)
ac_reconstructed = ac_dec(ac_embedding)

# construct discriminator: regular
dsc_input_dim = (WINDOW_SIZE, 1)
dsc_input = Input(shape = input_dim)
dsc_struct = discriminator_structure(dsc_input_dim)
dsc_label = dsc_struct(dsc_input)
ac_dsc_label = dsc_struct(ac_reconstructed)


    
# ------------------------------------------------------------------
# PARZEN ENTROPY ESTIMATION
# ------------------------------------------------------------------
# the Parzen kernel is a zero-centered gaussian with bin-width standard deviation
std = (1.0 / (NBINS - 1))
norm = 1.0 / math.sqrt(2.0 * 3.14159 * std * std)
den = (2.0 * std * std)

def parzen_kernel(x):
    num = K.square(x)
    return norm * K.exp(-num / den)

# we use 10,000 samples to create our entropy estimate
N = 10000
log_2 = math.log(2.0)
bins = K.variable(np.linspace(-1.0, 1.0, NBINS))
r_bins = K.repeat_elements(bins.reshape((NBINS, 1)), N, 1)

# we increase the weight of the entropy loss over time while
# training
entropy_weight = K.variable(0.0, name = 'entropy_weight')
max_entropy_weight = 1.0
entropy_weight_rate = 0.1

def entropy_estimate(placeholder, code):
    # if there are less than N samples in this batch, we just use however much data
    # we have
    flt = K.flatten(code)
    end_idx = K.minimum(flt.shape[0], N)
    
    ref = flt[:end_idx]
    r_ref = K.repeat_elements(ref.reshape((1, end_idx)), NBINS, 0)

    r_kern = parzen_kernel(r_ref - r_bins[:, :end_idx])
    r_kern = K.sum(r_kern, axis = 1)
    r_kern /= K.sum(r_kern)

    # clip to a low value so the log doesn't underflow
    ent = K.clip(r_kern, 1e-9, 1.0)
    ent = -K.sum(ent * K.log(ent) / log_2)
    return ent#ent * entropy_weight

# ------------------------------------------------------------------
# WGAN-ESQUE OBJECTIVES
# ------------------------------------------------------------------
def minimize_value(y_true, y_pred):
    return y_pred

def maximize_value(y_true, y_pred):
    return -y_pred



# compile model
loss_weights = [300.0, 1.0]
loss_functions = ['mae', 'binary_crossentropy']
n_recons = 1
n_discrim = 1
n_code = 0
assert(n_recons + n_discrim + n_code == len(loss_weights))
assert(len(loss_weights) == len(loss_functions))

def opti():
    return Adam(lr = 5e-4)
    
autoencoder = Model(input = [ac_input], output = [ac_reconstructed])
autoencoder.compile(loss = 'mae', optimizer = opti())

make_trainable(autoencoder, False)
discriminator = Model(input = [dsc_input], output = [dsc_label])
discriminator.compile(loss = ['binary_crossentropy'], optimizer = opti())
discriminator.summary()

autoencoder.summary()

make_trainable(discriminator, False)
make_trainable(autoencoder, True)
model = Model(input = [ac_input], output = [ac_reconstructed] * n_recons + \
                                           [ac_dsc_label] * n_discrim + \
                                           [ac_embedding] * n_code)
model.compile(loss = loss_functions,
              loss_weights = loss_weights,
              optimizer = opti())
model.summary()



X_train = np.copy(processedWindows)
ntrain = X_train.shape[0]
discrim_epoch = False


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 512, 1)            0         
_________________________________________________________________
model_3 (Model)              (None, 1)                 187841    
Total params: 187,841.0
Trainable params: 187,841.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 512, 1)            0         
_________________________________________________________________
model_1 (Model)              (None, 128)               101281    
_________________________________________________________________
model_2 (Model)              (None, 512, 1)            115681    
Total params: 216,962.0
Trainable params: 216,962.0
Non-trainabl

In [21]:
# interleave numpy arrays of the same size along the first axis
def interleave(arr):    
    num = len(arr)
    
    r = np.empty(arr[0].shape)
    r = np.repeat(r, num, axis = 0)
    
    for i in xrange(0, num):
        r[i::num] = arr[i]
    
    return r

In [22]:
# interface to PESQ evaluation
def run_pesq(clean, to_eval):
    pesq_regex = re.compile("\(MOS-LQO\):  = ([0-9]+\.[0-9]+)")
    
    pesq_out = os.popen("./PESQ +16000 +wb " + clean + " " + to_eval).read()
    regex_result = pesq_regex.search(pesq_out)
    
    if (regex_result is None):
        return 0.0
    else:
        return float(regex_result.group(1))

In [23]:
def create_discrim_pairs(autoencoder, batch):
    num = batch.shape[0]
    generated = autoencoder.predict(batch)
    
    #'''
    X = interleave([batch, generated])
    y = interleave([np.ones(num), np.zeros(num)])
    
    return X, y
    #'''
    
    '''
    pairs = []
    labels = []
    
    for i in xrange(0, num):
        batch_sample = batch[i, :, :]
        gen_sample = generated[i, :, :]

        # "real" sample comes first
        pairs += [[batch_sample, gen_sample]]
    
    pairs = np.array(pairs)
    pairs = [pairs[:, 0, :, :], pairs[:, 1, :, :]]
    
    return pairs, np.ones(num)
    '''

In [24]:
def test_discriminator(discriminator, autoencoder, orig_samples, verbose = True):
    X, y = create_discrim_pairs(autoencoder, orig_samples)
    
    # verify discriminator was trained properly
    y_hat = discriminator.predict(X)
    
    #dist = np.mean(y_hat)
    
    #'''
    y_hat[y_hat >= 0.5] = 1
    y_hat[y_hat < 0.5] = 0
    
    n_total = y.shape[0]
    n_correct = np.sum(np.ravel(y_hat) == y)

    acc = n_correct * 100.0 / n_total
    #'''
    
    if (verbose):
        print "Discriminator evaluation: %0.02f"%(acc)
    return acc

In [25]:
def autoencoderTest(waveFilename, prefix, autoencoder, verbose = True, deleteFile = False):
    [rate, data] = sciwav.read(waveFilename)
    data = data.astype(np.float32)
    processedWave, wparams = preprocessWaveform(data)
    windows = extractWindows(processedWave, STEP_SIZE, OVERLAP_SIZE)
    

    # first, write desired reconstruction
    transformed, tparams = preprocessWindows(windows)
    if (verbose):
        print transformed.shape
    
    desired = unpreprocessWindows(transformed, tparams)
    desired = reconstructFromWindows(desired, OVERLAP_SIZE, OVERLAP_FUNC)
    desired = unpreprocessWaveform(desired, wparams)
    desired = np.clip(desired, -32767, 32767)
    
    # then, run NN on transformed windows
    transformed, tparams = preprocessWindows(windows)
    
    transformed = np.reshape(transformed, (transformed.shape[0], WINDOW_SIZE, 1))
    autoencOutput = autoencoder.predict(transformed, batch_size = 128, verbose = (1 if verbose else 0))
    if (verbose):
        print autoencOutput.shape
    autoencOutput = np.reshape(autoencOutput, (autoencOutput.shape[0], WINDOW_SIZE))
    
    if (verbose):
        print autoencOutput.shape
    recons = unpreprocessWindows(autoencOutput, tparams)
    recons = reconstructFromWindows(recons, OVERLAP_SIZE, OVERLAP_FUNC)
    recons = unpreprocessWaveform(recons, wparams)
    recons = np.clip(recons, -32767, 32767)
    
    outFilename = prefix + "_output.wav"
    sciwav.write(outFilename, rate, recons.astype(np.int16))
    
    pesq = run_pesq(waveFilename, outFilename)
    if (deleteFile):
        os.system("rm " + outFilename)
    
    metrics = [
        np.max(desired),
        np.min(desired),
        np.max(recons),
        np.min(recons),
        mse(recons, desired),
        avgErr(recons, desired),
        pesq
    ]
    
    if (verbose):
        print "Max/min desired:", metrics[0], metrics[1]
        print "Max/min recons: ", metrics[2], metrics[3]
        print waveFilename, " mse: ", metrics[4]
        print waveFilename, " avg err: ", metrics[5]
        print "PESQ", metrics[6]
        
    return metrics

In [26]:
autoencoderTest("./SA1.wav", "./train_output/SA1_res_uninit_", autoencoder)

(112, 512)
112/112 [==============================] - 0s
(112, 512, 1)
(112, 512)
Max/min desired: 4899.0 -4013.0
Max/min recons:  2784.53 -2495.1
./SA1.wav  mse:  25173.9
./SA1.wav  avg err:  71.8671
PESQ 1.111


[4899.0, -4013.0, 2784.5332, -2495.0959, 25173.934, 71.867096, 1.111]

In [27]:
np.set_printoptions(formatter={'float_kind':'{:4f}'.format})

BATCH_SIZE = 128
NUM_BATCHES = ntrain / BATCH_SIZE
NUM_EPOCHS = 200

DSC_CLIP_WEIGHTS = False
DSC_CLAMP_RANGE = 0.01
DSC_TIMES_TRAIN = 1

lead = "    "
d_loss = 0.0
a_losses = []
d_acc = 0.0

train_auto = True

for epoch in range(NUM_EPOCHS):
    print "Epoch " + str(epoch + 1) + ":"

    # present batches randomly each epoch
    lis = range(0, ntrain, BATCH_SIZE)
    random.shuffle(lis)
    
    # keep track of start time and current batch #
    i = 0
    startTime = time.time()
    for idx in lis:
        batch = X_train[idx:idx+BATCH_SIZE, :,  :]
        nbatch = batch.shape[0]
        
        a_losses = ["autoencoder not training"]
        d_loss = "discriminator not training"

        
        # train autoencoder ("generator")
        make_trainable(autoencoder, True)
        make_trainable(discriminator, False)

        a_y = [batch] * n_recons + \
              [np.ones(nbatch)] * n_discrim + \
              [np.zeros(nbatch)] * n_code
        if (train_auto):
            a_losses = model.train_on_batch(batch, a_y)
        
        #if (i > 50):
        #    train_auto = False
        
        if (n_discrim > 0):
            # train discriminator
            make_trainable(autoencoder, False)
            make_trainable(discriminator, True)
            
            # clip discriminator weights, if necessary
            if (DSC_CLIP_WEIGHTS):
                for l in discriminator.layers:
                    weights = l.get_weights()
                    weights = [np.clip(w, -DSC_CLAMP_RANGE, DSC_CLAMP_RANGE) for w in weights]
                    l.set_weights(weights)
            
            discrim_batch_X, discrim_batch_y =  create_discrim_pairs(autoencoder, batch)
            for k in xrange(0, DSC_TIMES_TRAIN):
                d_loss = discriminator.train_on_batch(discrim_batch_X, discrim_batch_y)
        
        # print statistics every 10 batches so we know stuff is still going down
        if (i % 10 == 0):
            printStr = "        \r" + lead + str(i * BATCH_SIZE) + ": " + str(d_loss) + " "
            print printStr,
            
            loss_arr = np.asarray(a_losses)
            print loss_arr,
            
            if (len(loss_weights) > 1 and len(loss_arr) > 1):
                for w in xrange(0, len(loss_weights)):
                    loss_arr[w + 1] *= loss_weights[w]
                print loss_arr,
            
        i += 1
    print ""
    
    # print elapsed time for epoch
    elapsed = time.time() - startTime
    print lead + "Total time for epoch: " + str(elapsed) + "s"   
    
    # ---------------------------------------------------------
    # evaluate discriminator on random samples every epoch
    # ---------------------------------------------------------
    startTime = time.time()
    print lead + "----------------"
    if (n_discrim > 0):
        NUM = 200
        rows = np.random.randint(X_train.shape[0], size = NUM)
        d_acc = test_discriminator(discriminator, autoencoder,
                                   X_train[rows, :], verbose = False)

        print lead + "Evaluated the discriminator: " + str(d_acc)
        elapsed = time.time() - startTime
        print lead + "Total time for evaluation: " + str(elapsed) + "s"
    else:
        print lead + "No discriminator"
    
    
    # ---------------------------------------------------------
    # generate code histogram from said random samples
    # ---------------------------------------------------------
    NUM = 200
    rows = np.random.randint(X_train.shape[0], size = NUM)
    code = ac_enc.predict(X_train[rows, :], verbose = 0)
    
    print lead + "----------------"
    print lead + "Code histogram:"
    scalars = code.flatten()
    
    b = np.linspace(-1.0, 1.0, NBINS + 1)
    hist = np.histogram(scalars, bins = b)
    sample_hist_probs = hist[0].astype('float32')
    sample_hist_bins = hist[1].astype('float32')
    sample_hist_probs /= np.sum(sample_hist_probs)

    entropy = 0
    for i in sample_hist_probs:
        if (i < 1e-4): continue
        entropy += i * math.log(i, 2)
    entropy = -entropy
    
    zero_prob = sample_hist_probs[NBINS / 2]
    zero_prob = np.clip(zero_prob, 0.001, 0.999)
    mask_entropy = -(zero_prob * math.log(zero_prob, 2) + (1.0 - zero_prob) * math.log(1.0 - zero_prob, 2))
    
    print "       Entropy:", entropy
    print "       Zero prob:", sample_hist_probs[NBINS / 2]
    print "       Mask entropy:", mask_entropy
    print "       Pct. in last bins:", sample_hist_probs[0] + sample_hist_probs[-1]
    
    nnz = 0.0
    for i in xrange(0, code.shape[0]):
        r = np.round(code[i] * 1000.0) / 1000.0
        nnz += np.count_nonzero(r)
    nnz /= code.shape[0]
    print "       Avg # nonzero elts:", nnz
    
    # ---------------------------------------------------------
    # evaluate autoencoder on real data evey epoch
    # ---------------------------------------------------------
    startTime = time.time()
    print lead + "----------------"
    
    print lead + "Evaluating autoencoder..."
    metrics = autoencoderTest("./SA1.wav", "./train_output/SA1_res_reg_train_epoch" + str(epoch+1),
                              autoencoder, verbose = False)
    
    print lead + "Max/min desired:", metrics[0], metrics[1]
    print lead + "Max/min recons: ", metrics[2], metrics[3]
    print lead + "MSE:        ", metrics[4]
    print lead + "Avg err:    ", metrics[5]
    print lead + "PESQ:       ", metrics[6]
    
    metrics_tst = autoencoderTest("./SX383.wav", "./train_output/SX383_res_reg_train_epoch" + str(epoch+1),
                                  autoencoder, verbose = False, deleteFile = True)
    print lead + "PESQ (tst): ", metrics_tst[6]
    
    elapsed = time.time() - startTime
    print lead + "Total time for evaluation: " + str(elapsed) + "s"
    
    # half the learning rate at increments of 50 epochs
    if ((epoch + 1) % 50 == 0):
        old_rate = model.optimizer.lr.get_value()
        new_rate = old_rate / 2.0
        model.optimizer.lr.set_value(new_rate)
        
        print lead + "Changed learning rate from", old_rate, "to", new_rate
    
    # ---------------------------------------------------------
    # update entropy loss weight every epoch
    # ---------------------------------------------------------
    if (n_code > 0 and (epoch + 1) >= 5):
        v = K.get_value(entropy_weight)
        
        if (v < max_entropy_weight):
            v += entropy_weight_rate
            print lead + "Updated entropy constraint weight:", v
        else:
            v = max_entropy_weight
            print lead + "Didn't update entropy constraint weight:", v

        K.set_value(entropy_weight, v)

Epoch 1:
    6400: 0.632445216179  [4.426277 0.011257 1.049137] [4.426277 3.377140 1.049137]

KeyboardInterrupt: 

In [31]:
model.save('model_res_reg.h5')
autoencoder.save('auto_res_reg.h5')

discriminator.save('discrim_res_reg.h5')

import h5py

f = h5py.File('model_res_reg.h5', 'r+')
del f['optimizer_weights']
f.close()


In [32]:
from keras.models import load_model

objs = {'PhaseShiftUp1D' : PhaseShiftUp1D,
        'LinearUpSampling1D' : LinearUpSampling1D,
        'CodeRound' : CodeRound,
        'NBINS' : NBINS}

model = load_model('model_res_reg.h5', objs)
autoencoder = load_model('auto_res_reg.h5', objs)
discriminator = load_model('discrim_res_reg.h5', objs)

In [ ]:
enc = model.layers[1].layers
dec = model.layers[2].layers

In [ ]:
dsc_layers = discriminator.layers[1].layers

print "--- Discriminator layers ---"

i = 0
for l in dsc_layers:
    if type(l) is Convolution1D or type(l) is Dense:
        i += 1
        if type(l) is Convolution1D:
            print "Conv layer", i
        else:
            print "Dense layer", i
        w = l.weights[0].eval()
        print "    Avg weight norm:", np.mean(np.abs(w))
        print "    Max weight norm:", np.max(np.abs(w))
        
        if (len(l.weights) == 1): continue
        b = l.weights[1].eval()
        print "    Avg bias norm:", np.mean(np.abs(b))
        print "    Max bias norm:", np.max(np.abs(b))

In [ ]:
NUM = 400
rows = np.random.randint(X_train.shape[0], size = NUM)

d_acc = test_discriminator(discriminator, autoencoder,
                           X_train[rows, :], verbose = True)

In [ ]:
autoencoderTest("./SA1.wav", "SA1_final_", autoencoder)
autoencoderTest("./SX383.wav", "SX383_final_", autoencoder)
autoencoderTest("./fiveYears.wav", "fy_final_", autoencoder)

In [ ]:
all_embed = ac_enc.predict(X_train[:10000], batch_size = BATCH_SIZE, verbose = 1)

In [ ]:
scalars = all_embed.flatten()
log_scalars = np.log((scalars + 1.0) / 2.0)

In [ ]:
print np.mean(scalars)
print np.var(scalars)

In [ ]:
hist = np.histogram(scalars, bins = np.linspace(-1.0, 1.0, NBINS + 1))
sample_hist_probs = hist[0].astype('float32')
sample_hist_bins = hist[1].astype('float32')
sample_hist_probs /= np.sum(sample_hist_probs)

sample_hist_width = 1 * (sample_hist_bins[1] - sample_hist_bins[0])
sample_hist_centers = (sample_hist_bins[:-1] + sample_hist_bins[1:]) / 2
plt.bar(sample_hist_centers, sample_hist_probs, align='center', width=sample_hist_width)
plt.show()

entropy = 0
for i in sample_hist_probs:
    if (i < 1e-4): continue
    entropy += i * math.log(i, 2)
entropy = -entropy
print "Entropy of distribution:", entropy

In [ ]:
[rate, data] = sciwav.read("./SA1.wav")
data = data.astype(np.float32)
processedWave, wparams = preprocessWaveform(data)
windows = extractWindows(processedWave, STEP_SIZE, OVERLAP_SIZE)

transformed, tparams = preprocessWindows(windows)

transformed = np.reshape(transformed, (transformed.shape[0], WINDOW_SIZE, 1))
embed = ac_enc.predict(transformed, batch_size = BATCH_SIZE, verbose = 1)

In [ ]:
recons = ac_dec.predict(embed, batch_size = BATCH_SIZE, verbose = 1)
recons = unpreprocessWindows(recons, tparams)
#recons = reconstructFromWindows(recons, OVERLAP_SIZE, OVERLAP_FUNC)
#recons = unpreprocessWaveform(recons, wparams)
#recons = np.clip(recons, -32767, 32767)

In [ ]:
idx = 44
print np.count_nonzero(embed[idx]), "nonzero"
print embed[idx]

print len(scalars)
print np.count_nonzero((abs(scalars) > 1).astype('int'))

In [ ]:
idx = 56

orig = windows[idx].flatten()
recn = recons[idx].flatten()

plt.plot(orig)
ylim = plt.gca().get_ylim()
plt.show()

plt.plot(recn)
plt.ylim(ylim)
plt.show()

plt.plot(abs(orig - recn))
plt.show()